In [ ]:
import pandas as pd
import urllib.parse

import re
import numpy as np

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
# !pip install "tensorflow-gpu>=1.13.1,<2.0"

In [ ]:
# !pip install ampligraph 

In [ ]:
import ampligraph
ampligraph.__version__

'1.4.0'

In [ ]:
triples = pd.read_csv('/content/drive/MyDrive/bkmg_project/Integrated_KG_CSV.csv')

In [ ]:
triples = [tuple(x) for x in triples.to_records(index=False)]

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
triples_df = pd.DataFrame(triples, columns=["subject", "predicate", "object"])


In [ ]:
triples_df

,subject,predicate,object
0,http://example.org/MovieDB/Pete's_Dragon,http://www.w3.org/2002/07/owl#sameAs,http://example.org/MovieLens/160573
1,http://example.org/MovieDB/The_Forbidden_Dance,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,https://schema.org/Movie
2,http://example.org/MovieDB/GoldenEye,http://example.org/StreamingOn,http://example.org/MovieDB/Hulu
3,http://example.org/MovieLens/68954,http://example.org/RatedBy,http://example.org/MovieLens/User_Id_246
4,http://example.org/MovieDB/Into_the_Woods,https://schema.org/datePublished,2014
...,...,...,...
57228,http://example.org/MovieDB/Oliver_Hudson,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,https://schema.org/actor
57229,http://example.org/MovieDB/102_Dalmatians,http://example.org/DirectedBy,http://example.org/MovieDB/Kevin_Lima
57230,http://example.org/MovieDB/Bill_Thompson,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,https://schema.org/actor
57231,http://example.org/MovieDB/Marcus_Hester,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,https://schema.org/actor


In [ ]:
from ampligraph.evaluation import train_test_split_no_unseen 

X_train, X_valid = train_test_split_no_unseen(np.array(triples), test_size=5000)

In [ ]:
from ampligraph.latent_features import ComplEx,TransE

#kept no of epochs very low for testing
complex_model = ComplEx(batches_count=100,
                epochs=500,
                k=150,
                eta=20,
                optimizer='adam', 
                optimizer_params={'lr':1e-4},
                loss='multiclass_nll',
                regularizer='LP', 
                regularizer_params={'p':3, 'lambda':1e-5}, 
                seed=0, 
                verbose=True)


transe_model = TransE(k=150, 
                      eta=30, 
                      epochs=500, 
                      batches_count=100, 
                      seed=0, 
                      optimizer='adam', 
                      optimizer_params={'lr': 1e-4}, 
                      loss='multiclass_nll', 
                      regularizer='LP', 
                      regularizer_params={'p':3, 'lambda':1e-5},
                      verbose=True)

In [ ]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

complex_model.fit(X_train)
transe_model.fit(X_train)


Average TransE Loss:   0.465097: 100%|██████████| 500/500 [06:09<00:00,  1.35epoch/s]


In [ ]:
filter_triples = np.concatenate((X_train, X_valid))
from ampligraph.evaluation import evaluate_performance

ranks = evaluate_performance(X_valid,
                             model=complex_model, 
                             filter_triples=filter_triples,
                             use_default_protocol=True,
                             verbose=True)

WARNING - DeprecationWarning: use_default_protocol will be removed in future. Please use corrupt_side argument instead.


100%|██████████| 5000/5000 [00:54<00:00, 91.98it/s]


In [ ]:
from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score

mr = mr_score(ranks)
mrr = mrr_score(ranks)

print("MRR: %.2f" % (mrr))
print("MR: %.2f" % (mr))

hits_10 = hits_at_n_score(ranks, n=10)
print("Hits@10: %.2f" % (hits_10))
hits_3 = hits_at_n_score(ranks, n=3)
print("Hits@3: %.2f" % (hits_3))
hits_1 = hits_at_n_score(ranks, n=1)
print("Hits@1: %.2f" % (hits_1))

MRR: 0.18
MR: 1080.00
Hits@10: 0.29
Hits@3: 0.19
Hits@1: 0.13


In [ ]:
filter_triples = np.concatenate((X_train, X_valid))
from ampligraph.evaluation import evaluate_performance

ranks = evaluate_performance(X_valid,
                             model=transe_model, 
                             filter_triples=filter_triples,
                             use_default_protocol=True,
                             verbose=True)

WARNING - DeprecationWarning: use_default_protocol will be removed in future. Please use corrupt_side argument instead.


100%|██████████| 5000/5000 [00:35<00:00, 139.04it/s]


In [ ]:
from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score

mr = mr_score(ranks)
mrr = mrr_score(ranks)

print("MRR: %.2f" % (mrr))
print("MR: %.2f" % (mr))

hits_10 = hits_at_n_score(ranks, n=10)
print("Hits@10: %.2f" % (hits_10))
hits_3 = hits_at_n_score(ranks, n=3)
print("Hits@3: %.2f" % (hits_3))
hits_1 = hits_at_n_score(ranks, n=1)
print("Hits@1: %.2f" % (hits_1))

MRR: 0.24
MR: 529.81
Hits@10: 0.36
Hits@3: 0.26
Hits@1: 0.17
